# Ray RLlib - Multi-Armed Bandits - Exercise Solutions

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../../../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

Let's explore a very simple contextual bandit example with 3 arms. We'll run trials using RLlib and [Tune](http://tune.io), Ray's hyperparameter tuning library. 

In [1]:
import gym
from gym.spaces import Discrete, Box
import numpy as np
import pandas as pd
import os, time, random
import ray
from ray.tune.progress_reporter import JupyterNotebookReporter

In [2]:
!../../../tools/start-ray.sh --check --verbose

INFO: Ray is already running.


In [3]:
ray.init(address='auto', ignore_reinit_error=True)

{'node_ip_address': '192.168.1.105',
 'raylet_ip_address': '192.168.1.105',
 'redis_address': '192.168.1.105:6379',
 'object_store_address': '/tmp/ray/session_2020-07-18_20-05-29_369775_4270/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-07-18_20-05-29_369775_4270/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-07-18_20-05-29_369775_4270'}

## 03: Simple Multi-Armed Bandits - Exercise 1

## Exercise 1

Change the the `step` method to randomly change the `current_context` on each invocation:

```python
def step(self, action):
    result = super().step(action)
    self.current_context = random.choice([-1.,1.])
    return (np.array([-self.current_context, self.current_context]), reward, True,
            {
                "regret": 10 - reward
            })
```

Repeat the training and analysis. Does the training behavior change in any appreciable way? Why or why not?

See the [solutions notebook](solutions/Multi-Armed-Bandits-Solutions.ipynb) for discussion of this and the following exercises.

In [4]:
class SimpleContextualBandit2 (gym.Env):
    def __init__ (self, config=None):
        self.action_space = Discrete(3)     # 3 arms
        self.observation_space = Box(low=-1., high=1., shape=(2, ), dtype=np.float64)  # Random (x,y), where x,y from -1 to 1
        self.current_context = None
        self.rewards_for_context = {
            -1.: [-10, 0, 10],
            1.: [10, 0, -10],
        }

    def reset (self):
        self.current_context = random.choice([-1., 1.])
        return np.array([-self.current_context, self.current_context])

    def step (self, action):
        reward = self.rewards_for_context[self.current_context][action]
        self.current_context = random.choice([-1.,1.])
        return (np.array([-self.current_context, self.current_context]), reward, True,
                {
                    "regret": 10 - reward
                })

    def __repr__(self):
        return f'SimpleContextualBandit2(action_space={self.action_space}, observation_space={self.observation_space}, current_context={self.current_context}, rewards per context={self.rewards_for_context})'
    

In [5]:
bandit = SimpleContextualBandit2()
observation = bandit.reset()
f'Initial observation = {observation}, bandit = {repr(bandit)}'

'Initial observation = [ 1. -1.], bandit = SimpleContextualBandit2(action_space=Discrete(3), observation_space=Box(2,), current_context=-1.0, rewards per context={-1.0: [-10, 0, 10], 1.0: [10, 0, -10]})'

In [6]:
stop = {
    "training_iteration": 200,
    "timesteps_total": 100000,
    "episode_reward_mean": 10.0,
}

config = {
    "env": SimpleContextualBandit2,
}

In [7]:
analysis = ray.tune.run("contrib/LinUCB", config=config, stop=stop, 
    progress_reporter=JupyterNotebookReporter(overwrite=False),  # This is the default, actually.
    verbose=1)

/home/paul/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/paul/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/paul/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/paul/anaconda3/e

Trial name,status,loc
contrib_LinUCB_SimpleContextualBandit2_b6ad7_00000,RUNNING,


(pid=17169) /home/paul/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=17169)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=17169) /home/paul/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=17169)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=17169) /home/paul/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=17169)  

2020-07-19 00:41:08,934	ERROR trial_runner.py:520 -- Trial contrib_LinUCB_SimpleContextualBandit2_b6ad7_00000: Error processing event.
Traceback (most recent call last):
  File "/home/paul/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 468, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "/home/paul/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/ray/tune/ray_trial_executor.py", line 430, in fetch_result
    result = ray.get(trial_future[0], DEFAULT_GET_TIMEOUT)
  File "/home/paul/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/ray/worker.py", line 1474, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(AttributeError): ray::LinUCB.train() (pid=17169, ip=192.168.1.105)
  File "python/ray/_raylet.pyx", line 446, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 400, in ray._raylet.execute_task.function_executor
  File "/home/paul/anaconda3/envs/anysca

Trial name,status,loc
contrib_LinUCB_SimpleContextualBandit2_b6ad7_00000,ERROR,
Trial name,# failures,error file
contrib_LinUCB_SimpleContextualBandit2_b6ad7_00000,1,/home/paul/ray_results/contrib/LinUCB/contrib_LinUCB_SimpleContextualBandit2_0_2020-07-19_00-41-07gyn53xz_/error.txt


TuneError: ('Trials did not complete', [contrib_LinUCB_SimpleContextualBandit2_b6ad7_00000])

(pid=17278) /home/paul/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=17278)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=17278) /home/paul/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=17278)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=17278) /home/paul/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=17278)  

In [ ]:
stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

In [ ]:
df = analysis.dataframe()
df

It trains just as easily as the original implementation that didn't switch contexts between steps. Is this surprising? Probably not, because the relationship between the reward and the context remains linear, so what LinUCB learns for one context is correct for the second context, too. Also, _Tune_ runs many episodes, so it studies both contexts.

## 03: Simple Multi-Armed Bandits - Exercise 2

Recall the `rewards_for_context` we used:

```python
self.rewards_for_context = {
    -1.: [-10, 0, 10],
    1.: [10, 0, -10],
}
```

We said that Linear Upper Confidence Bound assumes a linear dependency between the expected reward of an action and its context. It models the representation space using a set of linear predictors.

Change the values for the rewards as follows, so they no longer have the same simple linear relationship:

```python
self.rewards_for_context = {
    -1.: [-10, 10, 0],
    1.: [0, 10, -10],
}
```

Also remove the change made for exercise 1, the line `self.current_context = random.choice([-1.,1.])` in the `step` method.

Run the training again and look at the results for the reward mean in TensorBoard. How successful was the training? How smooth is the plot for `episode_reward_mean`? How many steps were taken in the training?

In [ ]:
class SimpleContextualBanditNonlinear (gym.Env):
    def __init__ (self, config=None):
        self.action_space = Discrete(3)     # 3 arms
        self.observation_space = Box(low=-1., high=1., shape=(2, ), dtype=np.float64)  # Random (x,y), where x,y from -1 to 1
        self.current_context = None
        self.rewards_for_context = {   # Changed here:
            -1.: [-10, 10, 0],
            1.: [0, 10, -10],
        }

    def reset (self):
        self.current_context = random.choice([-1., 1.])
        return np.array([-self.current_context, self.current_context])

    def step (self, action):
        reward = self.rewards_for_context[self.current_context][action]
        return (np.array([-self.current_context, self.current_context]), reward, True,
                {
                    "regret": 10 - reward
                })

    def __repr__(self):
        return f'SimpleContextualBanditNonlinear(action_space={self.action_space}, observation_space={self.observation_space}, current_context={self.current_context}, rewards per context={self.rewards_for_context})'

In [ ]:
bandit = SimpleContextualBanditNonlinear()
observation = bandit.reset()
f'Initial observation = {observation}, bandit = {repr(bandit)}'

In [ ]:
print(f'current_context = {bandit.current_context}')
for i in range(10):
    action = bandit.action_space.sample()
    observation, reward, done, info = bandit.step(action)
    print(f'observation = {observation}, action = {action}, reward = {reward:4d}, done = {str(done):5s}, info = {info}')

In [ ]:
# `stop` defined above is unchanged.

config = {
    "env": SimpleContextualBanditNonlinear,
}

In [ ]:
analysis = ray.tune.run("contrib/LinUCB", config=config, stop=stop, 
    progress_reporter=JupyterNotebookReporter(overwrite=False),  # This is the default, actually.
    verbose=1)

In [ ]:
stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

In [ ]:
df = analysis.dataframe()
df

It ran the maximum of 20,000 steps and the best it does falls between 4.8 to 5.2 (for different runs), not 10.0. the `episode_reward_mean` is chaotic:

![Nonlinear model with LinUCB](../../../images/rllib/TensorBoard2.png).

Because LinUCB expcts a linear relationship between the context and each reward, it's not surprising that it fails to converge to the desired reward mean.

## 03: Simple Multi-Armed Bandits - Exercise 3

We briefly discussed another algorithm for selecting the next action, _Thompson Sampling_, in the [previous lesson](../02-Exploration-vs-Exploitation-Strategies.ipynb). Repeat exercises 1 and 2 using linear version, called _Linear Thompson Sampling_ ([RLlib documentation](https://docs.ray.io/en/latest/rllib-algorithms.html?highlight=greedy#linear-thompson-sampling-contrib-lints)). To make this change, look at this code we used above:

```python
analysis = tune.run("contrib/LinUCB", config=config, stop=stop, 
                    progress_reporter=JupyterNotebookReporter(overwrite=False),  # This is the default, actually.
                    verbose=2)  # Change to 0 or 1 to reduce the output.
```

Change `contrib/LinUCB` to `contrib/LinTS`.  

In [ ]:
bandit = SimpleContextualBandit2()
observation = bandit.reset()

# `stop` defined above is unchanged.

config = {
    "env": SimpleContextualBandit2,
}

analysis = ray.tune.run("contrib/LinTS", config=config, stop=stop, 
    progress_reporter=JupyterNotebookReporter(overwrite=False),  # This is the default, actually.
    verbose=1)

stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

In [ ]:
df = analysis.dataframe()
df

As before, the training only takes 200 steps and converge to the desired reward mean of `10.0`.

Now let's try the nonlinear bandit:

In [ ]:
bandit = SimpleContextualBanditNonlinear()
observation = bandit.reset()

# `stop` defined above is unchanged.

config = {
    "env": SimpleContextualBanditNonlinear,
}

start_time = time.time()

analysis = ray.tune.run("contrib/LinTS", config=config, stop=stop, 
                        progress_reporter=JupyterNotebookReporter(overwrite=False),  # This is the default, actually.
                        verbose=1)

print("The trials took", time.time() - start_time, "seconds\n")

In [ ]:
df = analysis.dataframe()
df

This run with Thompson sampling yields similar results with the reward mean between 4.5 and 5.0, with somewhat chaotic results over the 20000 steps, if you look at the `episode_reward_mean` graph in TensorBoard.

## 04: Linear Upper Confidence Bound - Exercise 1

Change the `training_iterations` from 20 to 40. Does the characteristic behavior of cumulative regret change at higher steps?

In [ ]:
from ray.rllib.contrib.bandits.agents.lin_ucb import UCB_CONFIG
from ray.rllib.contrib.bandits.envs import ParametricItemRecoEnv

In [ ]:
UCB_CONFIG["env"] = ParametricItemRecoEnv

# Actual training_iterations will be 40 * timesteps_per_iteration (100 by default) = 4,000
training_iterations = 40

print("Running training for %s time steps" % training_iterations)

In [ ]:
analysis = ray.tune.run(
    "contrib/LinUCB",
    config=UCB_CONFIG,
    stop={"training_iteration": training_iterations},
    num_samples=5,
    checkpoint_at_end=False,
    verbose=1
)

stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

In [ ]:
frame = pd.DataFrame()

for key, df in analysis.trial_dataframes.items():
    frame = frame.append(df, ignore_index=True)

df = frame.groupby("info/num_steps_trained")[
    "info/learner/default_policy/cumulative_regret"].aggregate(["mean", "max", "min", "std"])
df

In [ ]:
import sys
sys.path.append('../../..') # ... and line_plot functions from "util"
from util.line_plots import plot_line, plot_line_with_stddev, plot_line_with_min_max

sys.path.append("..")          # So we can load the bokeh_util from the parent directory...
from bokeh_util import plot_cumulative_regret, plot_wheel_bandit_model_weights

In [ ]:
# The next two lines prevent Bokeh from opening the graph in a new window.
import bokeh
bokeh.io.reset_output()
bokeh.io.output_notebook()

In [ ]:
plot_cumulative_regret(df)

([image](../../../images/rllib/LinUCB-cumulative-regret2.png))

The slope appears to stop flattening, suggesting that the previous number of steps, 2000, was sufficient to get the optimal behavior. Beyond that, regret continues to accumulate, but it's linear in the number of steps, neither getting better or worse.  

## 05: Linear Thompson Sampling - Exercise 1

Experiment with different $\delta$ values, for example 0.7 and 0.9. What do the cumulative regret and weights graphs look like? 

You can set the $\delta$ value like this:

```python
TS_CONFIG["delta"] = 0.7
```

In [ ]:
from ray.rllib.contrib.bandits.agents import LinTSTrainer
from ray.rllib.contrib.bandits.agents.lin_ts import TS_CONFIG
from ray.rllib.contrib.bandits.envs import WheelBanditEnv

In [ ]:
TS_CONFIG["env"] = WheelBanditEnv

training_iterations = 20
print("Running training for %s time steps" % training_iterations)

In [ ]:
def run_ts(delta):
    TS_CONFIG["delta"] = delta

    start_time = time.time()

    analysis = ray.tune.run(
        LinTSTrainer,
        config=TS_CONFIG,
        stop={"training_iteration": training_iterations},
        num_samples=2,
        checkpoint_at_end=True,
        verbose=1)

    print("The trials took", time.time() - start_time, "seconds\n")

    df = pd.DataFrame()

    for key, df_trial in analysis.trial_dataframes.items():
        df = df.append(df_trial, ignore_index=True)

    return df, analysis

In [ ]:
def process_df(df, analysis):
    ts_regrets = df \
        .groupby("info/num_steps_trained")["info/learner/default_policy/cumulative_regret"] \
        .aggregate(["mean", "max", "min", "std"])
    
    trial = analysis.trials[0]
    trainer = LinTSTrainer(config=TS_CONFIG)
    trainer.restore(trial.checkpoint.value)
    
    model = trainer.get_policy().model
    means = [model.arms[i].theta.numpy() for i in range(5)]
    covs = [model.arms[i].covariance.numpy() for i in range(5)]

    return ts_regrets, model, means, covs

In [ ]:
delta = 0.7
ts_df7, analysis7 = run_ts(delta)

In [ ]:
ts_regrets7, model7, means7, covs7 = process_df(ts_df7, analysis7)

In [ ]:
plot_cumulative_regret(ts_regrets7)

([image](../../../images/rllib/LinTS-Cumulative-Regret-07.png))

The cumulative regret values are much higher than for $\delta = 0.5$ in the lesson, and the standard deviation is ... well crazy. We mentioned in the lesson that the problem becomes harder for higher $\delta$, which fits this result.

In [ ]:
plot_wheel_bandit_model_weights(means7, covs7)

([image](../../../images/rllib/LinTS-Weight-Distribution-of-Arms-07.png))

Compare to the separation of the clusters compared to $\delta = 0.5$:

![image](../../../images/rllib/LinTS-Weight-Distribution-of-Arms-05.png)


In [ ]:
delta = 0.9
ts_df9, analysis9 = run_ts(delta)

In [ ]:
ts_regrets9, model9, means9, covs9 = process_df(ts_df9, analysis9)

In [ ]:
plot_cumulative_regret(ts_regrets9)

([image](../../../images/rllib/LinTS-Cumulative-Regret-09.png))

Qualitatively the same as for $\delta = 0.7$, but the size of the cumulative regret values are even higher. 

In [ ]:
plot_wheel_bandit_model_weights(means9, covs9)

([image](../../../images/rllib/LinTS-Weight-Distribution-of-Arms-09.png))

## 06 Market Example - Exercise 1

Try using a `LinUCBTrainer`-based trainer. How does the annualized return compare?

In [ ]:
# Some properties we'll need:
DEFAULT_MAX_INFLATION = 100.0
DEFAULT_TICKERS = ["sp500", "t.bill", "t.bond", "corp"]
DEFAULT_DATA_FILE = os.path.abspath(os.path.curdir) + '/../market.tsv'  # full path

def load_market_data (file_name):
    with open(file_name, "r") as f:
        return pd.read_table(f)

In [ ]:
df = load_market_data(DEFAULT_DATA_FILE)
df

In [ ]:
n_years = len(df)

In [ ]:
from ray.rllib.agents.trainer import with_base_config, with_common_config
from ray.rllib.contrib.bandits.agents.lin_ucb import UCB_CONFIG
from ray.rllib.contrib.bandits.agents.lin_ucb import LinUCBTrainer
import ray

In [ ]:
class MarketBandit (gym.Env):

    def __init__ (self, config={}):
        self.max_inflation = config.get('max-inflation', DEFAULT_MAX_INFLATION)
        self.tickers = config.get('tickers', DEFAULT_TICKERS)
        self.data_file = config.get('data-file', DEFAULT_DATA_FILE)
        print(f"MarketBandit: max_inflation: {self.max_inflation}, tickers: {self.tickers}, data file: {self.data_file} (config: {config})")

        self.action_space = Discrete(4)
        self.observation_space = Box(
            low  = -self.max_inflation,
            high =  self.max_inflation,
            shape=(1, )
        )
        self.df = load_market_data(self.data_file)
        self.cur_context = None


    def reset (self):
        self.year = self.df["year"].min()
        self.cur_context = self.df.loc[self.df["year"] == self.year]["inflation"][0]
        self.done = False
        self.info = {}

        return [self.cur_context]


    def step (self, action):
        if self.done:
            reward = 0.
            regret = 0.
        else:
            row = self.df.loc[self.df["year"] == self.year]

            # calculate reward
            ticker = self.tickers[action]
            reward = float(row[ticker])

            # calculate regret
            max_reward = max(map(lambda t: float(row[t]), self.tickers))
            regret = round(max_reward - reward)

            # update the context
            self.cur_context = float(row["inflation"])

            # increment the year
            self.year += 1

            if self.year >= self.df["year"].max():
                self.done = True

        context = [self.cur_context]
        #context = self.observation_space.sample()

        self.info = {
            "regret": regret,
            "year": self.year
        }

        return [context, reward, self.done, self.info]


    def seed (self, seed=None):
        """Sets the seed for this env's random number generator(s).
        Note:
            Some environments use multiple pseudorandom number generators.
            We want to capture all such seeds used in order to ensure that
            there aren't accidental correlations between multiple generators.
        Returns:
            list<bigint>: Returns the list of seeds used in this env's random
              number generators. The first value in the list should be the
              "main" seed, or the value which a reproducer should pass to
              'seed'. Often, the main seed equals the provided 'seed', but
              this won't be true if seed=None, for example.
        """
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

In [ ]:
market_config = with_base_config(UCB_CONFIG, {
    "env":           MarketBandit,
    'max-inflation': DEFAULT_MAX_INFLATION,
    'tickers':       DEFAULT_TICKERS,
    'data-file':     DEFAULT_DATA_FILE
})

stop = {
    "training_iteration": 100
}

In [ ]:
MarketLinUCBTrainer = LinUCBTrainer.with_updates(
    name="MarketLinUCBTrainer",
    default_config=market_config,      # Will be merged with Trainer.COMMON_CONFIG (rllib/agent/trainer.py)
    #default_policy=[somePolicyClass]  # If we had a policy...
)

In [ ]:
analysis = ray.tune.run(
    MarketLinUCBTrainer,
    config=market_config,
    stop=stop,
    num_samples=3,    
    checkpoint_at_end=True,
    verbose=1
)

In [ ]:
stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

In [ ]:
df_ts = pd.DataFrame()

for key, df_trial in analysis.trial_dataframes.items():
    df_ts = df_ts.append(df_trial, ignore_index=True)
    
df_ts.head()

In [ ]:
rewards = df_ts \
    .groupby("info/num_steps_trained")["episode_reward_mean"] \
    .aggregate(["mean", "max", "min", "std"])

rewards

In [ ]:
regrets = df_ts \
    .groupby("info/num_steps_trained")["info/learner/default_policy/cumulative_regret"] \
    .aggregate(["mean", "max", "min", "std"])

regrets

The results for _LinTS_ were ~570 for reward mean and the regret stayed under 10000. So, training with _LinUCB_ isn't as successful.

In [ ]:
plot_line_with_stddev(rewards, x_col='info/num_steps_trained', y_col='mean', stddev_col='std', 
                      title='Rewards vs. Steps', x_axis_label='step', y_axis_label='reward')

([image](../../../images/rllib/Market-Bandit-Rewards-vs-Steps-LinUCB.png))

In [ ]:
plot_cumulative_regret(regrets)

([image](../../../images/rllib/Market-Bandit-Cumulative-Regret-LinUCB.png))

What's the annualized return?

In [ ]:
print("{:5.2f}% optimized return annualized".format(max(rewards["mean"]) / n_years))

The result is likely the same as the completely random choices investigated in the lesson or worse!!

The market that we're modeling doesn't exhibit a linear relationship between the context, inflation in our case, and the rewards. Hence, it's not too surprising that a linear algorithm would fail to model the behavior perfectly. What's interesting here is that Thompson Sampling did a noticeably better job than Upper Confidence Bound.